**Note**: The graphs that we build are symmetric, but due to memory constraints, when we construct them, we will only keep the upper diagonal of the adjacency matrices.

In [1]:
import json
import numpy as np
import pandas as pd
import scipy.sparse
import pickle as pkl
from collections import defaultdict

### 1. Data loading

We first load the data and process it to form the graphs.

In [2]:
# Load data
data_file = 'data/recipes_with_nutritional_info_fixed_qty.json'
with open(data_file, 'r') as f:
    data = json.load(f)

In [3]:
# Print sample recipe
data[0]

{'fsa_lights_per100g': {'fat': 'green',
  'salt': 'green',
  'saturates': 'green',
  'sugars': 'orange'},
 'id': '000095fc1d',
 'ingredients': [{'text': 'yogurt, greek, plain, nonfat'},
  {'text': 'strawberries, raw'},
  {'text': 'cereals ready-to-eat, granola, homemade'}],
 'instructions': [{'text': 'Layer all ingredients in a serving dish.'}],
 'nutr_per_ingredient': [{'fat': 0.8845044000000001,
   'nrg': 133.80964,
   'pro': 23.110512399999998,
   'sat': 0.26535132,
   'sod': 81.64656,
   'sug': 7.348190400000001},
  {'fat': 0.46,
   'nrg': 49.0,
   'pro': 1.02,
   'sat': 0.023,
   'sod': 2.0,
   'sug': 7.43},
  {'fat': 7.415,
   'nrg': 149.25,
   'pro': 4.17,
   'sat': 1.207,
   'sod': 8.0,
   'sug': 6.04}],
 'nutr_values_per100g': {'energy': 81.12946131894766,
  'fat': 2.140139263515891,
  'protein': 6.914436593565536,
  'salt': 0.05597816738985967,
  'saturates': 0.36534716195613937,
  'sugars': 5.08634103436144},
 'partition': 'train',
 'quantity': [{'text': '8'}, {'text': '1'},

### 2. Ingredients identification

In [4]:
# List ingredient types
full_ingredient_names = defaultdict(dict)
for recipe in data:
    for ing in recipe['ingredients']:
        if not tuple(ing['text'].split(',')) in full_ingredient_names[ing['text'].split(',')[0]]:
            full_ingredient_names[ing['text'].split(',')[0]][tuple(ing['text'].split(','))] = 1
        else:
            full_ingredient_names[ing['text'].split(',')[0]][tuple(ing['text'].split(','))] += 1

To form the list of ingredients, we look for ingredient names that are specific enough. This means that we do not want one ingredient name to be associated to too many recipes nor to too few.

In [5]:
# Build trie for each ingredient class
class Trie:
    def __init__(self, val, d):
        self._build_trie(val, d)
        
    def _build_trie(self, val, d):
        if len(d) == 0:
            return
        self.val = val
        self.num = np.sum([d[k] for k in d])
        # Form next level
        children = defaultdict(dict)
        for k in d:
            if len(k) > 1:
                children[k[1]][k[1:]] = d[k]
        self.next = {}
        for k in children:
            self.next[k] = Trie(k, children[k])
        return self
    
    def relevant_ingredients(self, thresh, name, res):
        name.append(self.val)
        if len(self.next) <= 1:
            res.append((name, self.num))
            return
        # If all of the children have value below thresh, stop
        children_num = [self.next[child].num for child in self.next]
        if np.max(children_num) <= thresh:
            res.append((name, self.num))
            return
        for child in self.next:
            new_name = name.copy()
            self.next[child].relevant_ingredients(thresh, new_name, res)

In [6]:
# Retrieve the list of ingredients
thresh = len(data) / 10
ingredients_list = []
for ing in full_ingredient_names:
    t = Trie(ing, full_ingredient_names[ing])
    res = []
    t.relevant_ingredients(thresh, [], res)
    ingredients_list.extend(res)
ingredients = {''.join(ing[0]):ing[1] for ing in ingredients_list}

### 3. Ingredients and recipe data frames

We now organize dataframes for ingredients and recipes.

In [7]:
# Form ingredients dataframe
ingredients_df = pd.DataFrame({'name': [ing for ing in ingredients]})
ingredients_df.head()

,name
0,yogurt
1,strawberries
2,cereals ready-to-eat
3,sugars granulated
4,sugars powdered


In [8]:
# Identify ingredients in recipes and their weights
recipe_ingredients = []
for recipe in data:
    ings_list = [ing['text'].split(',') for ing in recipe['ingredients']]
    # Normalize ingredient weights
    weights = np.array(recipe['weight_per_ingr'])
    weights = 100 * weights / weights.sum()
    
    ings = []
    for i in range(len(ings_list)):
        ing = ings_list[i]
        ing_name = ing[0]
        for j in range(1, len(ing) + 1):
            if ing_name in ingredients:
                ings.append((ingredients_df[ingredients_df['name'] == ing_name].index[0], weights[i]))
                break
            ing_name += ing[j]
    recipe_ingredients.append(sorted(ings))

In [9]:
# Build dataframe
recipes_df = pd.DataFrame({
    'name': [d['title'] for d in data],
    'ingredients': recipe_ingredients,
    'energy': [d['nutr_values_per100g']['energy'] for d in data],
    'fat': [d['nutr_values_per100g']['fat'] for d in data],
    'protein': [d['nutr_values_per100g']['protein'] for d in data],
    'salt': [d['nutr_values_per100g']['salt'] for d in data],
    'saturates': [d['nutr_values_per100g']['saturates'] for d in data],
    'sugars': [d['nutr_values_per100g']['sugars'] for d in data]
})

# Merge together identical ingredients
def merge_ingredients(recipe):
    ings = recipe['ingredients']
    
    new_ings, v, w = [], ings[0][0], 0
    for i in range(len(ings)):
        if ings[i][0] != v:
            new_ings.append((v, w))
            v, w = ings[i][0], 0
        w += ings[i][1]
    new_ings.append((v, w))
    
    recipe['ingredients'] = new_ings
    return recipe

recipes_df = recipes_df.apply(merge_ingredients, axis=1)

# Free memory occupied by data
del data

recipes_df.head()

,name,ingredients,energy,fat,protein,salt,saturates,sugars
0,Yogurt Parfaits,"[(0, 55.41124271920566), (1, 37.13693757085337...",81.129461,2.140139,6.914437,0.055978,0.365347,5.086341
1,"Salt Free, Low Cholesterol Sugar Cookies Recipe","[(3, 13.479389772334713), (6, 22.4656496205578...",477.096404,23.412486,7.625492,0.548621,3.425054,14.298443
2,Honey Sriracha Chicken Wings,"[(7, 0.8547499191680942), (15, 86.157272296732...",208.058983,14.297046,15.383456,1.063915,4.535687,3.048951
3,Shrimp and Caper Salad,"[(7, 6.859617037531474), (53, 57.6197668757032...",194.752596,15.980767,11.946687,0.614843,2.366704,0.314583
4,Natural Peanut Butter Chocolate Bon Bons,"[(50, 36.17929562433298), (60, 9.1782283884738...",457.097118,29.329776,14.049093,0.029883,6.382497,35.608324


In [10]:
# Save the dataframes in a pickle
with open('data/dataframes.pkl', 'wb') as f:
    pkl.dump({
        'ingredients': ingredients_df,
        'recipes': recipes_df
    }, f)

### 4. Ingredient graph

We now form the ingredients graph. The weight of an edge will be equal to the number of recipes the pair of ingredients share.

In [11]:
ingredients_graph = np.zeros((len(ingredients_df), len(ingredients_df)))
for i in range(len(recipes_df)):
    ings = recipes_df.iloc[i]['ingredients']
    for j in range(len(ings) - 1):
        for k in range(j + 1, len(ings)):
            ingredients_graph[ings[j][0], ings[k][0]] += 1

In [12]:
# Store ingredients graph
with open('data/ingredient_graph.pkl', 'wb') as f:
    pkl.dump(ingredients_graph, f)

### 5. Recipe graph

When associating ingredients to the recipes, the ones that appear in more than 10% of the recipes are considered common ingredients. These will not be taken into account when constructing the graph.

In [13]:
# Count occurences of each ingredient
ingredient_counts = np.zeros(len(ingredients_df))
for ind in recipes_df.index:
    ings = recipes_df.iloc[ind]['ingredients']
    ingredient_counts[[ing[0] for ing in ings]] += 1

# Extract relevant ingredients for each recipe
thresh = len(recipes_df) / 10
relevant_ingredients = recipes_df.apply(lambda x:
                        list(filter(lambda ing: ingredient_counts[ing[0]] <= thresh, x['ingredients'])), axis=1)

In [14]:
# Group recipes by ingredients
recipes_by_ingredients = defaultdict(list)
for ind in recipes_df.index:
    ings = relevant_ingredients.iloc[ind]
    for ing in ings:
        recipes_by_ingredients[ing[0]].append(ind)

The graph generation will be done in C++ due to memory and time constraints.

In [15]:
# Write recipes by ingredients as string
str_data = f'{len(recipes_df)} {len(recipes_by_ingredients)}\n'
for ing in recipes_by_ingredients:
    str_data += f"{len(recipes_by_ingredients[ing])}\n{' '.join([str(r) for r in recipes_by_ingredients[ing]])}\n"

with open('data/recipes_by_ingredients.txt', 'w') as f:
    f.write(str_data)

In [16]:
!g++ generate_recipes_graph.cpp; ./a.out; rm a.out

Finished ingredient with 2093 recipes in 1 seconds.
Finished ingredient with 1487 recipes in 1 seconds.
Finished ingredient with 111 recipes in 0 seconds.
Finished ingredient with 1225 recipes in 0 seconds.
Finished ingredient with 645 recipes in 0 seconds.
Finished ingredient with 2410 recipes in 2 seconds.
Finished ingredient with 1596 recipes in 1 seconds.
Finished ingredient with 921 recipes in 0 seconds.
Finished ingredient with 2693 recipes in 3 seconds.
Finished ingredient with 4814 recipes in 7 seconds.
Finished ingredient with 1720 recipes in 1 seconds.
Finished ingredient with 1875 recipes in 1 seconds.
Finished ingredient with 408 recipes in 0 seconds.
Finished ingredient with 4133 recipes in 5 seconds.
Finished ingredient with 1131 recipes in 0 seconds.
Finished ingredient with 108 recipes in 0 seconds.
Finished ingredient with 209 recipes in 0 seconds.
Finished ingredient with 2739 recipes in 3 seconds.
Finished ingredient with 4419 recipes in 6 seconds.
Finished ingredien

Finished ingredient with 152 recipes in 0 seconds.
Finished ingredient with 131 recipes in 0 seconds.
Finished ingredient with 87 recipes in 0 seconds.
Finished ingredient with 77 recipes in 0 seconds.
Finished ingredient with 130 recipes in 0 seconds.
Finished ingredient with 125 recipes in 0 seconds.
Finished ingredient with 125 recipes in 0 seconds.
Finished ingredient with 40 recipes in 0 seconds.
Finished ingredient with 192 recipes in 0 seconds.
Finished ingredient with 2 recipes in 0 seconds.
Finished ingredient with 66 recipes in 0 seconds.
Finished ingredient with 69 recipes in 0 seconds.
Finished ingredient with 258 recipes in 0 seconds.
Finished ingredient with 148 recipes in 0 seconds.
Finished ingredient with 129 recipes in 0 seconds.
Finished ingredient with 48 recipes in 0 seconds.
Finished ingredient with 202 recipes in 0 seconds.
Finished ingredient with 41 recipes in 0 seconds.
Finished ingredient with 38 recipes in 0 seconds.
Finished ingredient with 36 recipes in 0 

We obtained a adjacency lists which we would like to convert to a sparse matrix. However, the amount of data is very large (text file of 2G of edges) and it may not fit into memory. We therefore need to perform some sampling of the edges.

In [17]:
recipe_graph_file = 'data/recipe_graph.txt'

edge_values = []
with open(recipe_graph_file, 'r') as f:
    for i in range(len(recipes_df)):
        edges = f.readline().split(' ')[1:-1]
        edges_val = [float(edge[1:-1].split(',')[1]) for edge in edges]
        edge_values += edges_val

In [18]:
perc90 = np.percentile(edge_values, 90)
print(f'The number of edges is {len(edge_values)}.')
print(f'The smallest edge value is {np.min(edge_values)}.')
print(f'The largest edge value is {np.max(edge_values)}.')
print(f'The 90th percentile of the edge values is {perc90}.')

The number of edges is 126544955.
The smallest edge value is 0.173615.
The largest edge value is 8.89827.
The 90th percentile of the edge values is 1.5766.


Let's then sample the 10% most significant edges.

In [19]:
recipe_graph = scipy.sparse.lil_matrix((len(recipes_df), len(recipes_df)))
with open(recipe_graph_file, 'r') as f:
    for i in range(len(recipes_df)):
        raw_edges = f.readline().split(' ')[1:-1]
        edges = [(int(edge[1:-1].split(',')[0]), float(edge[1:-1].split(',')[1])) for edge in raw_edges]
        edges = list(filter(lambda x: x[1] >= perc90, edges))
        recipe_graph[i, [edge[0] for edge in edges]] = [edge[1] for edge in edges]
recipe_graph = scipy.sparse.csr_matrix(recipe_graph)

In [20]:
# Save graph
scipy.sparse.save_npz('data/recipe_graph.npz', recipe_graph)

In [21]:
print(f'The resulting number of edges is {recipe_graph.count_nonzero()}.')

The resulting number of edges is 12785429.
